In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/stat/rushing/season/2024/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [2]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [3]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [4]:
stats_columns = ['Position', 'Games Played', 'Rushing Attempts', 'Rushing Yards',
                 'Avg Yards per Rush Attempt', 'Long Rushing', '20+ Yard Rushes', 'Rushign Touchdowns',
                 'Rushing Yards per Game', 'Rushing Fumbles', 'Rushing Fumbles Lost', 'Rushing First Downs']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs
0,RB,16,345,"2,005",5.8,72,17,13,125.3,2,1,82
1,RB,17,325,"1,921",5.9,87,19,16,113.0,3,1,94
2,RB,17,304,"1,456",4.8,37,5,14,85.6,1,0,82
3,RB,14,303,"1,431",4.7,70,11,11,102.2,3,1,71
4,RB,17,250,"1,412",5.6,70,13,16,83.1,0,0,70
...,...,...,...,...,...,...,...,...,...,...,...,...
328,WR,17,3,-4,-1.3,3,0,0,-0.2,0,0,1
329,WR,11,1,-5,-5.0,-5,0,0,-0.5,0,0,0
330,WR,12,1,-5,-5.0,-5,0,0,-0.4,0,0,0
331,WR,14,2,-6,-3.0,4,0,0,-0.4,0,0,0


In [5]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats['Year'] = 2024
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Saquon Barkley,PHI,RB,16,345,"2,005",5.8,72,17,13,125.3,2,1,82,2024
1,2,Derrick Henry,BAL,RB,17,325,"1,921",5.9,87,19,16,113.0,3,1,94,2024
2,3,Bijan Robinson,ATL,RB,17,304,"1,456",4.8,37,5,14,85.6,1,0,82,2024
3,4,Jonathan Taylor,IND,RB,14,303,"1,431",4.7,70,11,11,102.2,3,1,71,2024
4,5,Jahmyr Gibbs,DET,RB,17,250,"1,412",5.6,70,13,16,83.1,0,0,70,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,323,Jalen Nailor,MIN,WR,17,3,-4,-1.3,3,0,0,-0.2,0,0,1,2024
329,330,Cedric Tillman,CLE,WR,11,1,-5,-5.0,-5,0,0,-0.5,0,0,0,2024
330,330,Jha'Quan Jackson,TEN,WR,12,1,-5,-5.0,-5,0,0,-0.4,0,0,0,2024
331,332,George Pickens,PIT,WR,14,2,-6,-3.0,4,0,0,-0.4,0,0,0,2024


In [6]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Rushing Attempts,Rushing Yards,Avg Yards per Rush Attempt,Long Rushing,20+ Yard Rushes,Rushign Touchdowns,Rushing Yards per Game,Rushing Fumbles,Rushing Fumbles Lost,Rushing First Downs,Year
0,1,Saquon Barkley,PHI,RB,16,345,"2,005",5.8,72,17,13,125.3,2,1,82,2024
1,2,Derrick Henry,BAL,RB,17,325,"1,921",5.9,87,19,16,113.0,3,1,94,2024
2,3,Bijan Robinson,ATL,RB,17,304,"1,456",4.8,37,5,14,85.6,1,0,82,2024
3,4,Jonathan Taylor,IND,RB,14,303,"1,431",4.7,70,11,11,102.2,3,1,71,2024
4,5,Jahmyr Gibbs,DET,RB,17,250,"1,412",5.6,70,13,16,83.1,0,0,70,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,323,Jalen Nailor,MIN,WR,17,3,-4,-1.3,3,0,0,-0.2,0,0,1,2024
329,330,Cedric Tillman,CLE,WR,11,1,-5,-5.0,-5,0,0,-0.5,0,0,0,2024
330,330,Jha'Quan Jackson,TEN,WR,12,1,-5,-5.0,-5,0,0,-0.4,0,0,0,2024
331,332,George Pickens,PIT,WR,14,2,-6,-3.0,4,0,0,-0.4,0,0,0,2024


In [7]:
total_stats.to_csv("Rushing_2024.csv", index = False)
print("Data saved to Rushing_2024.csv")

Data saved to Rushing_2024.csv
